# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/LSTM/ver1/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    #print("error")
    #history = "error"
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [11]:
for machine_type in machine_types:
    history = run(machine_type)
    #with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    #    pickle.dump(history , file)

2020-12-02 03:29:05,866 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-12-02 03:29:05,867 - 00_train.py - INFO - MAKE DATA_LOADER
2020-12-02 03:29:06,741 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 15/15 [00:02<00:00,  7.09it/s]
2020-12-02 03:29:20,707 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:48886.603027, tr_rec:48802.989014, tr_eng:39.795819, tr_covd:15926.885300, val_AUC:0.519038, val_pAUC:0.502907
100%|██████████| 15/15 [00:02<00:00,  7.09it/s]
2020-12-02 03:29:31,537 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:48862.133362, tr_rec:48780.672546, tr_eng:40.756094, tr_covd:15477.066788, val_AUC:0.508213, val_pAUC:0.498846
100%|██████████| 15/15 [00:02<00:00,  7.20it/s]
2020-12-02 03:29:43,963 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:48825.692932, tr_rec:48741.753418, tr_eng:31.559600, tr_covd:16156.735870, val_AUC:0.494954, val_pAUC:0.499165
100%|██████████| 15/15 [00:02<00:00,  7.08it/s]
2020-12-02 03:29:56,462 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:48759.314941, tr_rec:48677.369934, tr_eng:22.601816, tr_covd:15936.980499, val_AUC:0.480657, val_pAUC:0.494504
100%|██████████| 15/15 [00:01<00:00,  8.22it/s]
2020-12-

100%|██████████| 15/15 [00:01<00:00,  8.98it/s]
2020-12-02 03:35:14,219 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:44721.005798, tr_rec:44740.149658, tr_eng:-270.490715, tr_covd:1581.071510, val_AUC:0.506310, val_pAUC:0.500038
100%|██████████| 15/15 [00:02<00:00,  6.87it/s]
2020-12-02 03:35:25,906 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:44340.757629, tr_rec:44359.308716, tr_eng:-263.532456, tr_covd:1560.433884, val_AUC:0.519437, val_pAUC:0.498546
100%|██████████| 15/15 [00:02<00:00,  7.21it/s]
2020-12-02 03:35:38,355 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:43942.376282, tr_rec:43963.234924, tr_eng:-285.288281, tr_covd:1534.055222, val_AUC:0.508658, val_pAUC:0.500047
100%|██████████| 15/15 [00:02<00:00,  6.93it/s]
2020-12-02 03:35:51,130 - pytorch_modeler.py - INFO - Epoch 33/300:train_loss:43528.169983, tr_rec:43549.492249, tr_eng:-288.872076, tr_covd:1513.002790, val_AUC:0.482092, val_pAUC:0.500994
100%|██████████| 15/15 [00:02<00:00,  7.06it/s]


100%|██████████| 15/15 [00:02<00:00,  6.77it/s]
2020-12-02 03:41:08,562 - pytorch_modeler.py - INFO - Epoch 59/300:train_loss:29100.503357, tr_rec:29121.504150, tr_eng:-275.454569, tr_covd:1308.964876, val_AUC:0.429187, val_pAUC:0.500239
100%|██████████| 15/15 [00:02<00:00,  7.03it/s]
2020-12-02 03:41:21,258 - pytorch_modeler.py - INFO - Epoch 60/300:train_loss:28444.906982, tr_rec:28469.956177, tr_eng:-310.812271, tr_covd:1206.381111, val_AUC:0.464471, val_pAUC:0.501894
100%|██████████| 15/15 [00:02<00:00,  7.16it/s]
2020-12-02 03:41:33,716 - pytorch_modeler.py - INFO - Epoch 61/300:train_loss:27803.719055, tr_rec:27830.664124, tr_eng:-329.079020, tr_covd:1192.548985, val_AUC:0.492400, val_pAUC:0.506030
100%|██████████| 15/15 [00:01<00:00, 10.10it/s]
2020-12-02 03:41:44,706 - pytorch_modeler.py - INFO - Epoch 62/300:train_loss:27210.429688, tr_rec:27231.697327, tr_eng:-274.111248, tr_covd:1228.690142, val_AUC:0.449733, val_pAUC:0.497721
100%|██████████| 15/15 [00:02<00:00,  6.88it/s]


...

100%|██████████| 7/7 [00:00<00:00, 11.08it/s]
2020-12-02 08:40:44,478 - pytorch_modeler.py - INFO - Epoch 205/300:train_loss:1284.524553, tr_rec:1271.601822, tr_eng:116.092789, tr_covd:262.690299, val_AUC:0.495934, val_pAUC:0.506160
100%|██████████| 7/7 [00:00<00:00, 11.43it/s]
2020-12-02 08:40:49,389 - pytorch_modeler.py - INFO - Epoch 206/300:train_loss:1227.648575, tr_rec:1214.815117, tr_eng:115.582159, tr_covd:255.046311, val_AUC:0.469033, val_pAUC:0.501793
100%|██████████| 7/7 [00:00<00:00, 10.79it/s]
2020-12-02 08:40:54,318 - pytorch_modeler.py - INFO - Epoch 207/300:train_loss:1209.252058, tr_rec:1198.196081, tr_eng:97.480823, tr_covd:261.579310, val_AUC:0.475063, val_pAUC:0.497961
100%|██████████| 7/7 [00:00<00:00, 11.00it/s]
2020-12-02 08:40:59,278 - pytorch_modeler.py - INFO - Epoch 208/300:train_loss:1184.136131, tr_rec:1173.628807, tr_eng:92.295751, tr_covd:255.549378, val_AUC:0.460544, val_pAUC:0.498467
100%|██████████| 7/7 [00:00<00:00, 10.60it/s]
2020-12-02 08:41:04,105 

100%|██████████| 7/7 [00:00<00:00, 11.03it/s]
2020-12-02 08:43:12,349 - pytorch_modeler.py - INFO - Epoch 235/300:train_loss:1122.751520, tr_rec:1109.876663, tr_eng:117.268437, tr_covd:229.602498, val_AUC:0.464368, val_pAUC:0.499798
100%|██████████| 7/7 [00:00<00:00, 11.05it/s]
2020-12-02 08:43:17,324 - pytorch_modeler.py - INFO - Epoch 236/300:train_loss:1150.954952, tr_rec:1138.168646, tr_eng:116.192047, tr_covd:233.421978, val_AUC:0.502451, val_pAUC:0.497340
100%|██████████| 7/7 [00:00<00:00, 10.96it/s]
2020-12-02 08:43:22,217 - pytorch_modeler.py - INFO - Epoch 237/300:train_loss:1158.823933, tr_rec:1146.511868, tr_eng:111.481422, tr_covd:232.785676, val_AUC:0.466846, val_pAUC:0.499451
100%|██████████| 7/7 [00:00<00:00, 11.08it/s]
2020-12-02 08:43:27,100 - pytorch_modeler.py - INFO - Epoch 238/300:train_loss:1086.284922, tr_rec:1073.928564, tr_eng:112.287173, tr_covd:225.526526, val_AUC:0.480272, val_pAUC:0.499841
100%|██████████| 7/7 [00:00<00:00, 10.91it/s]
2020-12-02 08:43:32,06

100%|██████████| 7/7 [00:00<00:00, 11.48it/s]
2020-12-02 08:45:35,743 - pytorch_modeler.py - INFO - Epoch 264/300:train_loss:1056.750340, tr_rec:1043.489515, tr_eng:121.781199, tr_covd:216.539905, val_AUC:0.482033, val_pAUC:0.499971
100%|██████████| 7/7 [00:00<00:00, 10.84it/s]
2020-12-02 08:45:40,707 - pytorch_modeler.py - INFO - Epoch 265/300:train_loss:1065.761944, tr_rec:1053.712994, tr_eng:109.489030, tr_covd:220.008104, val_AUC:0.472728, val_pAUC:0.497932
100%|██████████| 7/7 [00:00<00:00, 10.86it/s]
2020-12-02 08:45:45,699 - pytorch_modeler.py - INFO - Epoch 266/300:train_loss:1067.732182, tr_rec:1055.042927, tr_eng:116.014033, tr_covd:217.571723, val_AUC:0.485027, val_pAUC:0.499566
100%|██████████| 7/7 [00:00<00:00, 10.87it/s]
2020-12-02 08:45:50,641 - pytorch_modeler.py - INFO - Epoch 267/300:train_loss:1063.479736, tr_rec:1051.846527, tr_eng:105.467330, tr_covd:217.294350, val_AUC:0.491005, val_pAUC:0.501157
100%|██████████| 7/7 [00:00<00:00, 11.01it/s]
2020-12-02 08:45:55,56

100%|██████████| 7/7 [00:00<00:00, 10.74it/s]
2020-12-02 08:47:59,506 - pytorch_modeler.py - INFO - Epoch 293/300:train_loss:1039.966759, tr_rec:1027.993967, tr_eng:109.114905, tr_covd:212.260182, val_AUC:0.470945, val_pAUC:0.498149
100%|██████████| 7/7 [00:00<00:00, 11.09it/s]
2020-12-02 08:48:04,425 - pytorch_modeler.py - INFO - Epoch 294/300:train_loss:1035.758629, tr_rec:1024.212158, tr_eng:104.889652, tr_covd:211.499424, val_AUC:0.481591, val_pAUC:0.500564
100%|██████████| 7/7 [00:00<00:00, 10.99it/s]
2020-12-02 08:48:09,399 - pytorch_modeler.py - INFO - Epoch 295/300:train_loss:1064.994085, tr_rec:1052.183159, tr_eng:117.345603, tr_covd:215.273071, val_AUC:0.473604, val_pAUC:0.500896
100%|██████████| 7/7 [00:00<00:00, 11.22it/s]
2020-12-02 08:48:14,339 - pytorch_modeler.py - INFO - Epoch 296/300:train_loss:1043.583534, tr_rec:1032.051590, tr_eng:104.725439, tr_covd:211.879471, val_AUC:0.496195, val_pAUC:0.500622
100%|██████████| 7/7 [00:00<00:00, 10.96it/s]
2020-12-02 08:48:19,29

elapsed time: 1486.017235041 [sec]


history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)